# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [54]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [55]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [56]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056
[['', 'Logged In', 'Kevin', 'M', '0', 'Arellano', '', 'free', 'Harrisburg-Carlisle, PA', 'GET', 'Home', '1.54001E+12', '514', '', '200', '1.54207E+12', '66'], ['Fu', 'Logged In', 'Kevin', 'M', '1', 'Arellano', '280.05832', 'free', 'Harrisburg-Carlisle, PA', 'PUT', 'NextSong', '1.54001E+12', '514', 'Ja I Ty', '200', '1.54207E+12', '66'], ['', 'Logged In', 'Maia', 'F', '0', 'Burke', '', 'free', 'Houston-The Woodlands-Sugar Land, TX', 'GET', 'Home', '1.54068E+12', '510', '', '200', '1.54207E+12', '51'], ['All Time Low', 'Logged In', 'Maia', 'F', '1', 'Burke', '177.84118', 'free', 'Houston-The Woodlands-Sugar Land, TX', 'PUT', 'NextSong', '1.54068E+12', '510', 'A Party Song (The Walk of Shame)', '200', '1.54207E+12', '51'], ['Nik & Jay', 'Logged In', 'Wyatt', 'M', '0', 'Scott', '196.51873', 'free', 'Eureka-Arcata-Fortuna, CA', 'PUT', 'NextSong', '1.54087E+12', '379', 'Pop-Pop!', '200', '1.54208E+12', '9'], ["Quad City DJ's", 'Logged In', 'Chloe', 'F', '0', 'Cuevas', '451.44771', 'free

In [57]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [58]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [59]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)
except Exception as e:
    print(e)

#### Set Keyspace

In [60]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [61]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

## Table name: song_library
## Column 1: artist
## Column 2: itemInSession
## Column 3: length
## Column 4: sessionId
## Column 5: song
## PRIMARY KEY: (sessionId, itemInSession)
## As our query is looking for the sessionId and itemInSession, we can choose (sessionId, itemInSession) as composite primary key.
## Because sessionId is not unique, with itemInSession it will make Primary Key unique and distributed evenly. 


create_table_query = "CREATE TABLE IF NOT EXISTS song_library (artist text, itemInSession int, length float, sessionId int, song text, PRIMARY KEY(sessionId, itemInSession))"

try:
    session.execute(create_table_query)
except Exception as e:
    print(e)

In [62]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_library (artist, itemInSession, length, sessionId, song)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[0], int(line[3]), float(line[5]), int(line[8]), line[9]))

#### Do a SELECT to verify that the data have been inserted into each table

In [63]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
select_query = 'select artist, song as song_title, length as song_length from song_library where sessionId = 338 and itemInSession = 4'

try:
    rows = session.execute(select_query)
except Exception as e:
    print(e)

for row in rows:
    print('Artist:', row.artist, ', Song Title: ', row.song_title, ', Song Length: ', row.song_length)

Artist: Faithless , Song Title:  Music Matters (Mark Knight Dub) , Song Length:  495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [19]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

## Table name: user_info
## Column 1: artist_name
## Column 2: song_title
## Column 3: user_firstName
## Column 4: user_lastName
## Column 5: userid
## Column 6: sessionId
## Column 7: itemInSession
## PRIMARY KEY ((userId, sessionId), itemInSession)
## Using (userId, sessionId) as PRIMARY KEY and (itemInSession) as CLUSTERING COLUMN
## As our query is looking for userId and sessionId, I choose (userId, sessionId) as composite primary key. It is unique and evenly distributed.
## And because query wants the song sorted by itemInSession, I put itemInSession as Clustering Column, so the song will be sorted by that.

create_table_query2 = "CREATE TABLE IF NOT EXISTS user_info (artist_name text, song_title text, user_firstName text, user_lastName text, sessionId int, userId int, itemInSession int, PRIMARY KEY((userId, sessionId), itemInSession));"

try:
    session.execute(create_table_query2)
except Exception as e:
    print(e)                    

In [35]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query2 = "INSERT INTO user_info (artist_name, song_title, user_firstName, user_lastName, sessionId, userId, itemInSession) "
        query2 = query2 + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query2, (line[0], line[9], line[1], line[4], int(line[8]), int(line[10]), int(line[3])))

In [43]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
select_query2 = "select artist_name, song_title, user_firstName, user_lastName from user_info where userId = 10 and sessionId = 182"

try:
    rows = session.execute(select_query2)
except Exception as e:
    print(e)

for row in rows:
    print('Artist:', row.artist_name, ', Song: ', row.song_title, ', First Name:', row.user_firstname, ', Last Name:', row.user_lastname)

Artist: Down To The Bone , Song:  Keep On Keepin' On , First Name: Sylvie , Last Name: Cruz
Artist: Three Drives , Song:  Greece 2000 , First Name: Sylvie , Last Name: Cruz
Artist: Sebastien Tellier , Song:  Kilometer , First Name: Sylvie , Last Name: Cruz
Artist: Lonnie Gordon , Song:  Catch You Baby (Steve Pitron & Max Sanna Radio Edit) , First Name: Sylvie , Last Name: Cruz


In [46]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

## Table name: song_user
## Column 1: user_lastname
## Column 2: user_firstname
## Column 3: song_title
## Column 4: userId
## PRIMARY KEY (song_title, userId)
## As our query is looking for song_title, I choose song_title as primary key.
## However, song_title is not unique. Therefore, I put userId which is unique along with song_title to be a composite primary key. 
## This composite primary key will be unique and evenly distributed.

create_table_query3 = "CREATE TABLE IF NOT EXISTS song_user (user_firstname text, user_lastname text, song_title text, userId int, PRIMARY KEY(song_title, userId))"

try:
    session.execute(create_table_query3)
except Exception as e:
    print(e) 

In [47]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query3 = "INSERT INTO song_user (user_firstname, user_lastname, song_title, userId) "
        query3 = query3 + "VALUES (%s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query3, (line[1], line[4], line[9], int(line[10])))

In [50]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
select_query3 = "select user_firstname, user_lastname, song_title from song_user where song_title = 'All Hands Against His Own'"

try:
    rows = session.execute(select_query3)
except Exception as e:
    print(e)

for row in rows:
    print('User First Name:', row.user_firstname, ', User Last Name:', row.user_lastname)

User First Name: Jacqueline , User Last Name: Lynch
User First Name: Tegan , User Last Name: Levine
User First Name: Sara , User Last Name: Johnson


### Drop the tables before closing out the sessions

In [4]:
## TO-DO: Drop the table before closing out the sessions

In [51]:
query = 'drop table song_library'
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = 'drop table user_info'
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = 'drop table song_user'
try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [52]:
session.shutdown()
cluster.shutdown()